# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
from keras import backend as K

class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename)
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['text_cleaned'], float):
                texts.append(str(row['text_cleaned']))
            else:
                texts.append(row['text_cleaned'])

            labels.append(row['target'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = tf.keras.preprocessing.sequence.pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_testing_metric(self, y_test, y_pred):
        accuracyScore = accuracy_score(y_test, y_pred)
        precisionScore = precision_score(y_test, y_pred)
        recallScore = recall_score(y_test, y_test)
        f1Score = f1_score(y_test, y_pred)
        return accuracyScore, precisionScore, recallScore, f1Score

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_val_data(self, textsTraining, labelsTraining, train_index, val_index, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining)[train_index], np.asarray(textsTraining)[val_index]
        y_train, y_val = np.asarray(labelsTraining)[train_index], np.asarray(labelsTraining)[val_index]

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)

    def recall(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_score(self, y_true, y_pred):
        precision = self.precision(y_true, y_pred)
        recall = self.recall(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


# Finite State Machine

In [2]:
import random

def FSM():
    fsm = {}
    fsm[0] = {'src': 0, 'dst': 1, 'layer': 'embedding_layer', 'next_path': [1]}
    fsm[1] = {'src': 1, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 4]}
    fsm[2] = {'src': 2, 'dst': 3, 'layer': 'maxpooling_layer', 'next_path': [3]}    
    fsm[3] = {'src': 3, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 4]}
    fsm[4] = {'src': 2, 'dst': 4, 'layer': 'global_maxpooling_layer', 'next_path': [5]}
    fsm[5] = {'src': 4, 'dst': 5, 'layer': 'dense_layer', 'next_path': [6, 7]}
    fsm[6] = {'src': 5, 'dst': 5, 'layer': 'dense_layer', 'next_path': [6, 7]}    
    fsm[7] = {'src': 5, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [8]}
    fsm[8] = {'src': 6, 'dst': 7, 'layer': 'output_layer', 'next_path': []}

    return fsm

def getLayerSize(layer, conv_idx, dense_idx, dropout_idx, maxpooling_idx):
    if layer == 'convolutional_layer':
        conv_idx += 1
    elif layer == 'dense_layer':
        dense_idx += 1
    elif layer == 'dropout_layer':
        dropout_idx += 1
    elif layer == 'maxpooling_layer':
        maxpooling_idx += 1
    return conv_idx, dense_idx, dropout_idx, maxpooling_idx

def generateFSM(hyperparameters):
    fsm = FSM()

    idx = conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
    path = [fsm[idx]['layer']]
    while len(fsm[idx]['next_path']) != 0:
        idx = random.choice(fsm[idx]['next_path'])
        layer = fsm[idx]['layer']
        path.append(layer)
        conv_idx, dense_idx, dropout_idx, maxpooling_idx = getLayerSize(layer, conv_idx, dense_idx, dropout_idx,
                                                                        maxpooling_idx)

    hyperparameters['layers'] = path

# CNN

In [3]:
import tensorflow as tf

class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters'], indiv['kernel_size'],
                                        kernel_initializer=indiv['conv_init_mode'],
                                        activation=indiv['conv_activation_func'],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint']),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons'],
                                       kernel_initializer=indiv['dense_init_mode'],
                                       activation=indiv['dense_activation_func'],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint'])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate']))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size']))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0, 
                                          nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                           decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                            decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                            schedule_decay=0.004)

        util = utility()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[util.f1_score])

        return model

# Path

In [4]:
training_path = 'trainPreprocessed.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = '{}/Dataset/Disaster Tweets/'.format(root_path)
resultsPath = '{}/Server2/Disaster Tweets/Paper GA-CNN/Results/'.format(root_path)
archPath = '{}/Server2/Disaster Tweets/Paper GA-CNN/Architecture/'.format(root_path)
testing_name = "Experiment6_RandomSearch"
glovePath = ['{}/Glove/glove.6B.50d.txt'.format(root_path),
             '{}/Glove/glove.6B.100d.txt'.format(root_path),
             '{}/Glove/glove.6B.200d.txt'.format(root_path),
             '{}/Glove/glove.6B.300d.txt'.format(root_path)]

# Run

In [5]:
import time
import os
import csv
import tensorflow as tf
import random

# object class
util = utility()
cnn = CNN()

# Read data
df = util.read_CSV(datasetPath + training_path)

from sklearn.model_selection import train_test_split
dfTraining, dfVal = train_test_split(df, test_size = 0.3, random_state=42)

# get texts and labels
textsTraining, y_train = util.get_text_label(dfTraining)
textsVal, y_val = util.get_text_label(dfVal)

# Tokenize words
tokenizer = util.tokenize_texts(textsTraining)
X_train = tokenizer.texts_to_sequences(textsTraining)
X_val = tokenizer.texts_to_sequences(textsVal)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# get maxlen
maxlen = util.get_max_length_of_sentences(textsTraining)

# Pad sequences with zeros
X_train = util.padding_texts(X_train, maxlen)
X_val = util.padding_texts(X_val, maxlen)

tempF1Score=0
for i in range(0, 900):
    print(i)
    then = time.time()
    text = ''
    hyperparameters = {}
    
    # Optimized hyperparameters
    hyperparameters['epochs'] = random.randint(1, 100)
    hyperparameters['batch_size'] = random.randint(32, 256)
    hyperparameters['optimizer'] = random.choice(['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam'])
    hyperparameters['learning_rate'] = random.uniform(1e-4, 1e-2)
    hyperparameters['momentum'] = random.uniform(0, 1)
    hyperparameters['output_init_mode'] = random.choice(['zeros',
                              'ones',
                              'uniform',
                              'normal',
                              'glorot_normal',
                              'glorot_uniform',
                              'he_normal',
                              'he_uniform',
                              'lecun_normal',
                              'lecun_uniform'])
    hyperparameters['output_dim'] = random.choice([50, 100, 200, 300])
    hyperparameters["num_filters"] = random.randint(32, 512)
    hyperparameters["kernel_size"] = random.randint(1, 5)
    hyperparameters["conv_activation_func"] = random.choice(
                                               ['relu', 'softmax', 'elu', 'selu',
                                                'softplus', 'softsign', 'tanh',
                                                'sigmoid', 'hard_sigmoid', 'linear'])
    hyperparameters["conv_init_mode"] = random.choice(
                                         ['zeros',
                                          'ones',
                                          'uniform',
                                          'normal',
                                          'glorot_normal',
                                          'glorot_uniform',
                                          'he_normal',
                                          'he_uniform',
                                          'lecun_normal',
                                          'lecun_uniform'])
    hyperparameters["conv_weight_constraint"] = random.randint(1, 5)
    hyperparameters["neurons"] = random.randint(1, 30)
    hyperparameters["dense_activation_func"] = random.choice(
                                                ['relu', 'softmax', 'elu', 'selu',
                                                 'softplus', 'softsign', 'tanh',
                                                 'sigmoid', 'hard_sigmoid', 'linear'])
    hyperparameters["dense_init_mode"] = random.choice(
                                          ['zeros',
                                           'ones',
                                           'uniform',
                                           'normal',
                                           'glorot_normal',
                                           'glorot_uniform',
                                           'he_normal',
                                           'he_uniform',
                                           'lecun_normal',
                                           'lecun_uniform'])
    hyperparameters["dense_weight_constraint"] = random.randint(1, 5)
    hyperparameters["pool_size"] = random.randint(2, 6)
    hyperparameters["dropout_rate"] = random.uniform(0, 1)
    
    generateFSM(hyperparameters)

#     print(hyperparameters)

    if int(hyperparameters['output_dim']) == 50:
        glove = glovePath[0]
    elif int(hyperparameters['output_dim']) == 100:
        glove = glovePath[1]
    elif int(hyperparameters['output_dim']) == 200:
        glove = glovePath[2]
    elif int(hyperparameters['output_dim']) == 300:
        glove = glovePath[3]

    embedding_matrix = util.create_embedding_matrix(glove, tokenizer.word_index, int(hyperparameters['output_dim']))

    model = cnn.cnn_model(vocab_size, maxlen, embedding_matrix, hyperparameters, hyperparameters['layers'])

    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', verbose=False, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint("{}{}.h5".format(archPath, testing_name), monitor='val_f1_score', mode='max', verbose=False, save_best_only=True)]
       
#     model.summary()
    y_train = np.uint8(y_train)
    y_val = np.uint8(y_val)
    model.fit(X_train, y_train, epochs=hyperparameters['epochs'], verbose=False, validation_data=(X_val, y_val),
              batch_size=hyperparameters['batch_size'], callbacks=callbacks)

    dependencies = {
    'f1_score': util.f1_score
    }

    # load the saved model
    for x in range(0, 4):  # try 4 times
        try:
            # msg.send()
            saved_model = tf.keras.models.load_model("{}{}.h5".format(archPath, testing_name), custom_objects=dependencies)   
            str_error = None
        except Exception as e:
            print('An error occurs when loading saved model.')
            str_error = e
            pass

        if str_error:
            sleep(2)  # wait for 2 seconds before trying to fetch the data again
        else:
            break
    
    y_pred = saved_model.predict_classes(X_val)
    os.remove("{}{}.h5".format(archPath, testing_name))   

    # CNN metrics
    accuracyScore, precisionScore, recallScore, f1Score = util.get_testing_metric(y_val, y_pred)

    now = time.time()
    diff = now - then

    # save testing data
    f = open(resultsPath + testing_name + ".csv", 'a')
    text = "{0},".format('i')
    for param in hyperparameters:
        text += "{0},".format(param)
    text +='accuracy,precision,recall,f1Score,time\n'
    text += "{0}".format(str(i + 1))
    for value in hyperparameters.values():
        text += ",{0}".format(value)

    text += "{0},{1},{2},{3},{4},{5},{6}".format(',',accuracyScore, precisionScore, recallScore, f1Score, diff, '\n')
    f.write(text)    
    f.close()

    # save testing data
    f = open(resultsPath + testing_name + '_summary' + ".csv", 'a')
    text = "{0},{1},{2},{3},{4},{5}\n".format(str(i + 1),accuracyScore, precisionScore, recallScore, f1Score, diff)
    if f1Score > tempF1Score:
        tempF1Score = f1Score
        print(f1Score)
        text += "'Best F1Score is in iteration {0} with {1}\n".format(str(i + 1),tempF1Score)
    f.write(text)    
    f.close() 

441
442


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


443
444
445
446
447


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


448
449
450
451


...

895


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


896
897


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


898
899


In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>